In [ ]:
import cv2
import numpy as np
from picamera2 import Picamera2
from pyzbar.pyzbar import decode

In [ ]:
def load_calibration_params(file_path):
    try:
        calibration_data = np.load(file_path)
        camera_matrix = calibration_data['camera_matrix']
        dist_coeffs = calibration_data['dist_coeffs']
        return camera_matrix, dist_coeffs
    except Exception as e:
        print(f"Error loading calibration parameters: {e}")
        return None, None

In [ ]:
def undistort_image(image, camera_matrix, dist_coeffs):
    h, w = image.shape[:2]
    new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
    undistorted_image = cv2.undistort(image, camera_matrix, dist_coeffs, None, new_camera_matrix)
    return undistorted_image

In [ ]:
def detect_qr_code(frame):
    decoded_objects = decode(frame)
    if decoded_objects:
        for obj in decoded_objects:
            print(f"Detected QR Code: {obj.data}")
            return obj.data
    return None

In [ ]:
def authenticate_user(picam, camera_matrix, dist_coeffs):
    print("Esperando a que se presente el código QR del usuario...")

    while True:
        frame = picam.capture_array()
        undistorted_frame = undistort_image(frame, camera_matrix, dist_coeffs)
        cv2.imshow("picam", undistorted_frame)

        qr_data = detect_qr_code(undistorted_frame)
        if qr_data:
            print(f"Usuario autenticado. Código QR detectado: {qr_data}")
            break

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cv2.destroyAllWindows()

def stream_video(calibration_file):
    picam = Picamera2()
    picam.preview_configuration.main.size = (500, 300)
    picam.preview_configuration.main.format = "RGB888"
    picam.preview_configuration.align()
    picam.configure("preview")
    picam.start()

    # Cargar parámetros de calibración
    camera_matrix, dist_coeffs = load_calibration_params(calibration_file)

    if camera_matrix is None or dist_coeffs is None:
        return

    # Autenticar al usuario mediante el código QR
    authenticate_user(picam, camera_matrix, dist_coeffs)

    # Aquí puedes agregar el código para la función de la cinta

if __name__ == "__main__":
    calibration_file_path = "calibration_params.npz"
    stream_video(calibration_file_path)
